# Strip Surface and Bottom from Glorys Daily Data

In [1]:
####  Libraries  ####
import xarray as xr
import dask
import os
import numpy as np
import geopandas as gpd
import regionmask
import re

# Paths to resources
box_path = "/Users/adamkemberling/Library/CloudStorage/Box-Box/"
lobecol_path = f"{box_path}Mills Lab/Projects/Lobster ECOL/Spatial_Defs/"
glorys_path = f"{box_path}RES_Data/GLORYs/"


# Get file paths
fpaths_glorys = os.listdir(f"{glorys_path}NE_Shelf_TempSal/")


# File paths, no ds_store
# Pattern matching for .nc only
pattern = r".*\.nc$"  # Match all files ending with ".nc"

# list the .nc files
fpaths_glorys = []
for filename in os.listdir(f"{glorys_path}NE_Shelf_TempSal/"):
    if re.search(pattern, filename):
        fpaths_glorys.append(f"{glorys_path}NE_Shelf_TempSal/{filename}")



# Load all of the Glorys year as one file
# # Lazy-load the data itself using xr.open_mfdataset
glorys_all = xr.open_mfdataset(fpaths_glorys, combine = "by_coords", parallel = True)
glorys_all

<xarray.Dataset> Size: 213GB
Dimensions:    (time: 11762, depth: 38, latitude: 132, longitude: 226)
Coordinates:
  * depth      (depth) float32 152B 0.494 1.541 2.646 ... 1.245e+03 1.452e+03
  * latitude   (latitude) float32 528B 35.25 35.33 35.42 ... 46.0 46.08 46.17
  * longitude  (longitude) float32 904B -75.67 -75.58 -75.5 ... -57.0 -56.92
  * time       (time) datetime64[ns] 94kB 1993-01-01 1993-01-02 ... 2025-03-25
Data variables:
    thetao     (time, depth, latitude, longitude) float64 107GB dask.array<chunksize=(31, 38, 132, 226), meta=np.ndarray>
    so         (time, depth, latitude, longitude) float64 107GB dask.array<chunksize=(31, 38, 132, 226), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GLORYS12V1
    title:                     daily mean fields from Global Ocean Physics An...
    z_max:                     5727.9169921875
    z_min:                     0.49402499198913574
    copernicusmarine_version:  1.3.1

# Specify subset

This notebook is still at the moment a core step for building out the inventory of surface and bottom only data. But we don't need to re-run all years each time for new data. The next code chunk subsets to whichever range of years we need.

In [11]:
# Just 2025
glorys_all = glorys_all.sel(time = slice("2025-01-01", "2025-12-31"))


<xarray.Dataset> Size: 1GB
Dimensions:    (time: 81, depth: 38, latitude: 132, longitude: 226)
Coordinates:
  * depth      (depth) float32 152B 0.494 1.541 2.646 ... 1.245e+03 1.452e+03
  * latitude   (latitude) float32 528B 35.25 35.33 35.42 ... 46.0 46.08 46.17
  * longitude  (longitude) float32 904B -75.67 -75.58 -75.5 ... -57.0 -56.92
  * time       (time) datetime64[ns] 648B 2025-01-01 2025-01-02 ... 2025-03-25
Data variables:
    thetao     (time, depth, latitude, longitude) float64 735MB dask.array<chunksize=(31, 38, 132, 226), meta=np.ndarray>
    so         (time, depth, latitude, longitude) float64 735MB dask.array<chunksize=(31, 38, 132, 226), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GLORYS12V1
    title:                     daily mean fields from Global Ocean Physics An...
    z_max:                     5727.9169921875
    z_min:                     0.49402499198913574
    copernicusmarine_version:  1.3.1

### Getting Bottom Layer Information

Only need one file to get the bottom layer indexing information. This will pull the deepest non-NA value's index information. That can then be used to get the depth itself ("bottom depth") and be fed back to extract the temperature and salinity for that layer.

In [2]:
# Open one file to get depth indices
glorys_single = xr.open_mfdataset(fpaths_glorys[0], combine = "by_coords", parallel = True)


# Identify Bottom Indices
# find bottom temp for any netcdf with depth
def find_deepest_depth_indices(ds, variable_id, y_coord, x_coord, depth_coord, maxDepth = 2000):


    # Subset up to an optional max depth
    kwargs = {depth_coord: slice(0, maxDepth)}
    bottom_400 = ds.sel(**kwargs)

    # First get the vertical True/False of valid values
    idx = bottom_400[variable_id].isel(time=0).isnull()
    idx_vals = idx.values


    if len(bottom_400[variable_id][x_coord].dims) == 2:
        multiIndex = True
    else:
        multiIndex = False

    if multiIndex == True:
        dims0 = bottom_400[y_coord].dims[0]
        dims1 = bottom_400[y_coord].dims[1]
    else:
        dims0 = y_coord
        dims1 = x_coord


    # Create the initial final array to store indices (integer type)
    depth_indices = np.zeros((len(idx[y_coord][dims0]), len(idx[x_coord][dims1]))).astype(int)

    # Now find the deepest depth where values are True and store in indices array
    for i in range(len(bottom_400[dims1].values)):
        for j in range(len(bottom_400[dims0].values)):
            located = np.where(idx_vals[:, j, i] == False)
            try:
                depth_indices[j, i] = int(located[-1][-1])
            except IndexError:
                depth_indices[j, i] = 1

    # Gather as a DataArray
    ind = xr.DataArray(depth_indices, dims=[dims0, dims1])

    return ind


## Get Indices for "Bottom" Layer

Use the bottom-layer indexing function to pull the proper depth slices from the full array. This is what we'll use to extract temperature/salinity/depth data later.

In [3]:
# Pull the bottom indices from the single glorys file
glorys_bottom_idx = find_deepest_depth_indices(
    ds = glorys_single, 
    variable_id = 'thetao', 
    x_coord = 'longitude', 
    y_coord = 'latitude', 
    depth_coord = 'depth', 
    maxDepth = 2000)

## Pull Surface+Bottom Indices Across Time

This step creates two xr.datasets, one for the surface layer and the other for values at the bottom depth

In [4]:
# Refocus/Reshape surface and bottom datasets for masking
surface_ds = glorys_all.isel(depth = 0).rename_vars({"thetao" : "surface_temp", "so" : "surface_sal"})

# Use the bottom indices to extract the variables we care about

# use kwargs to pull values for those indices
kwdepth = {'depth': glorys_bottom_idx}
var_array = glorys_all#['thetao']

# Now index the values out
bottom_ds = var_array.isel(**kwdepth).rename_vars({"thetao" : "bottom_temp", "so" : "bottom_sal"})
bottom_ds

<xarray.Dataset> Size: 6GB
Dimensions:      (time: 11762, latitude: 132, longitude: 226)
Coordinates:
    depth        (latitude, longitude) float32 119kB 7.93 7.93 ... 55.76 55.76
  * latitude     (latitude) float32 528B 35.25 35.33 35.42 ... 46.0 46.08 46.17
  * longitude    (longitude) float32 904B -75.67 -75.58 -75.5 ... -57.0 -56.92
  * time         (time) datetime64[ns] 94kB 1993-01-01 1993-01-02 ... 2025-03-25
Data variables:
    bottom_temp  (time, latitude, longitude) float64 3GB dask.array<chunksize=(31, 132, 226), meta=np.ndarray>
    bottom_sal   (time, latitude, longitude) float64 3GB dask.array<chunksize=(31, 132, 226), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GLORYS12V1
    title:                     daily mean fields from Global Ocean Physics An...
    z_max:                     5727.9169921875
    z_min:                     0.49402499198913574
    copernicusmarine_version:  1.3.1

## Restructure to single xr.Dataset

Just reorganizing and renaming variables so that they can be joined easily later. The depth coordinate is removed and each variable is added back in so that we can pull any of the variables efficiently without fussing with depth.

The next function pulls what the bottom depth values were for each location, then stores it as a variable. This lets us map the bottom layer and double check down the line what the max depth was.

In [5]:
# Prepare the bottom dataset for resaving

# Drop the depth dimension where it exists, then rebuild.
# Takes the first layer for depth to grab surface measurements in the case of SST and bottom in the case where there
def depth_to_var(xr_ds, var, depth_var = "depth", t_coord = "time", x_coord = "longitude", y_coord = "latitude"):
    """Pull out data as an array, drop depth as a dimension, rebuild xr.array and add depth calues as a variable. 
    Need to pull surface measurement from surface data arrays so depth coordinate
    becomes unnecessary.
    
    Args:
        xr_ds      : xr.ArrayDataset
        var (str)  : String indicating variable to pull and process
    
    """
    
    # Pull the coordinates to keep
    time     = xr_ds.coords[t_coord]
    x_coords = xr_ds.coords[x_coord]
    y_coords = xr_ds.coords[y_coord]

    # Take the data values out as an array
    var_vals = xr_ds[var].values 
    depth_vals = xr_ds[depth_var].values

    # Rebuild an xr.array for the variable we're pulling out
    var_array = xr.DataArray(var_vals, coords = [time, y_coords, x_coords])

    # Another for depth
    depth_array = xr.DataArray(depth_vals, coords = [y_coords, x_coords])

    # Put them all together as one thing
    no_depth_ds = xr.Dataset({
        var      : var_array,
        "depth"  : depth_array})



    return no_depth_ds

In [6]:
# Do the reshaping
# moves depth into the variable
bottom_ds_new = depth_to_var(
    xr_ds = bottom_ds, 
    var = "bottom_temp", 
    depth_var = "depth", 
    t_coord = "time", 
    x_coord = "longitude", 
    y_coord = "latitude")

bottom_sal_new = depth_to_var(
    xr_ds = bottom_ds, 
    var = "bottom_sal", 
    depth_var = "depth", 
    t_coord = "time", 
    x_coord = "longitude", 
    y_coord = "latitude")

# inspect it
bottom_ds_new

<xarray.Dataset> Size: 3GB
Dimensions:      (time: 11762, latitude: 132, longitude: 226)
Coordinates:
  * time         (time) datetime64[ns] 94kB 1993-01-01 1993-01-02 ... 2025-03-25
  * latitude     (latitude) float32 528B 35.25 35.33 35.42 ... 46.0 46.08 46.17
  * longitude    (longitude) float32 904B -75.67 -75.58 -75.5 ... -57.0 -56.92
Data variables:
    bottom_temp  (time, latitude, longitude) float64 3GB 19.28 19.13 ... 0.5172
    depth        (latitude, longitude) float32 119kB 7.93 7.93 ... 55.76 55.76

## Re-combine Individual Arrays

It is easier to pull the components out and reassemble everything manually at this stage:

In [7]:
# Combine Surface and Bottom Variables into one dataset

# Pull the coordinates to keep
time     = surface_ds.coords["time"]
x_coords = surface_ds.coords["longitude"]
y_coords = surface_ds.coords["latitude"]

# Take the data values out as an array
sst_vals = surface_ds["surface_temp"].values 
sal_vals = surface_ds["surface_sal"].values
bsal_vals = bottom_sal_new["bottom_sal"].values

# Rebuild an xr.array for the variable we're pulling out
sst_array = xr.DataArray(sst_vals, coords = [time, y_coords, x_coords])
sal_array = xr.DataArray(sal_vals, coords = [time, y_coords, x_coords])
bsal_array = xr.DataArray(bsal_vals, coords = [time, y_coords, x_coords])

bottom_ds_new["surface_temp"] = sst_array
bottom_ds_new["surface_sal"] = sal_array
bottom_ds_new["bottom_sal"] = bsal_array
surfbot_ds = bottom_ds_new.rename({"depth" : "bottom_depth"})
surfbot_ds


<xarray.Dataset> Size: 11GB
Dimensions:       (time: 11762, latitude: 132, longitude: 226)
Coordinates:
  * time          (time) datetime64[ns] 94kB 1993-01-01 ... 2025-03-25
  * latitude      (latitude) float32 528B 35.25 35.33 35.42 ... 46.0 46.08 46.17
  * longitude     (longitude) float32 904B -75.67 -75.58 -75.5 ... -57.0 -56.92
Data variables:
    bottom_temp   (time, latitude, longitude) float64 3GB 19.28 19.13 ... 0.5172
    bottom_depth  (latitude, longitude) float32 119kB 7.93 7.93 ... 55.76 55.76
    surface_temp  (time, latitude, longitude) float64 3GB 18.72 19.29 ... 0.6044
    surface_sal   (time, latitude, longitude) float64 3GB 35.76 35.93 ... 31.98
    bottom_sal    (time, latitude, longitude) float64 3GB 35.94 35.84 ... 31.98

## Saving

These were originally saved locally, they could also be saved to Box at this step for convenience.

In [10]:
# Save destination file structure
surfbot_out = "../GLORYS_surfbot_temps/CMEMS_Northeast_TempSal_SurfaceBottom_"
glorys_path = f"{box_path}RES_Data/GLORYs/"
surfbot_box_out = f"{glorys_path}NE_Shelf_Surfbot_Daily/CMEMS_Northeast_TempSal_SurfaceBottom_"

# Group by year
for year, subset in surfbot_ds.groupby('time.year'):
    
    # Save each year's data to a separate NetCDF file
    subset.to_netcdf(f'{surfbot_out}{year}.nc')
    subset.to_netcdf(f"{surfbot_box_out}{year}.nc")